In [ ]:
##supervised learning 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D,MaxPooling2D

In [8]:
!nvidia-smi

Thu Apr  6 10:45:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.33       Driver Version: 528.33       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P0    16W /  50W |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [3]:
import os
import tensorflow as tf

In [12]:
##pip install tensorflow==2.12.*

In [9]:
df = pd.read_csv('train.csv')

In [10]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [16]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [18]:
df.iloc[1]['comment_text']

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [19]:
##Preprocess

In [20]:
!pip list

Package                           Version
--------------------------------- --------------------
absl-py                           1.4.0
aiofiles                          23.1.0
aiohttp                           3.8.4
aiosignal                         1.3.1
alabaster                         0.7.12
altair                            4.2.2
altair-data-server                0.4.1
altair-viewer                     0.4.0
anaconda-client                   1.11.0
anaconda-navigator                2.3.1
anaconda-project                  0.11.1
anyio                             3.5.0
appdirs                           1.4.4
apyori                            1.1.2
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.2
astroid                           2.11.7
astropy                           5.1
astunparse                        1.6.3
async-timeout                     4.0.2
atomicwrites                      1.4.0
attrs             

In [21]:
##tokenization(TextVectorization)
from tensorflow.keras.layers import TextVectorization

In [22]:
# Spliting the data into comment and features 
X = df['comment_text']
y = df[df.columns[2:]].values ##values will create numpy array

In [23]:
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [24]:
y 

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [25]:
MAX_FEATURES = 200000 # number of words in the vocab

In [26]:
## Creating TextVectorization layer
vectorizer = TextVectorization(max_tokens=MAX_FEATURES, ##passing Max_features
                               output_sequence_length=1800, ##This is sentences length in tokens 
                               output_mode='int') ##We want our output in integer...it goan map all comments in integer value 

In [27]:
vectorizer.adapt(X.values) ##adapt will learn all X.values 

In [28]:
vectorizer("Nice to meet you")

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 520,    3, 1007, ...,    0,    0,    0], dtype=int64)>

In [29]:
vectorizer("Nice to meet you")[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([ 520,    3, 1007,    7], dtype=int64)>

In [30]:
vectorizer_text = vectorizer(X.values) ## tokenizing all the text in int formate 

In [31]:
vectorizer_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [32]:
#creating tensorflow pipeline
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text, y)) ##creating a dataset ##input feature and y = traget freature 
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [33]:
batch_X , batch_y = dataset.as_numpy_iterator().next()

In [34]:
batch_X

array([[   399,      3,  35142, ...,      0,      0,      0],
       [    20,      7,   2947, ...,      0,      0,      0],
       [    64,     20,      7, ...,      0,      0,      0],
       ...,
       [ 36594,      2, 171479, ...,      0,      0,      0],
       [  8519,     34,      2, ...,      0,      0,      0],
       [    14,     15,     91, ...,      0,      0,      0]], dtype=int64)

In [35]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [36]:
int(len(dataset)*.7)

6981

In [37]:
train = dataset.take(int(len(dataset)*.7)) ##taking (assing)  70% to training
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) ##skipping 70% and using 20% for validation 
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))# skipping 90 % and 10 % for validation 

In [38]:
train_generator  = train.as_numpy_iterator()

In [39]:
train_generator.next()

(array([[   22,    92,  2681, ...,     0,     0,     0],
        [ 3975,    23,    18, ...,     0,     0,     0],
        [   94,    94,    13, ...,     0,     0,     0],
        ...,
        [   21,  1800,    12, ...,     0,     0,     0],
        [41322,  1958,     4, ...,     0,     0,     0],
        [    8,    74,   531, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [40]:
###Creating the sequentail model

In [41]:
#LSTM 
#Dropout : It is method of regularization 
#Bidirectional layer : it is allow us to pass the features or values  of LSTM layers acores the Sequences 
#Dense : it is fully conneted layer 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [42]:
model = Sequential()

In [43]:
#Creating the embedding layers
model.add(Embedding(MAX_FEATURES+1,32)) ##+1 for unknown word  ##32 features 

In [44]:
#creating the Bidirectional LSTM layer
model.add(Bidirectional(LSTM(32,activation='tanh'))) ##LSTM having 32 differernt units 
# we are using 'tanh' because for GPU acceleration we requrid ..and it define by tensorflow 

In [45]:
#creating the Feature extractor Fully conncected Layer 
model.add(Dense(128, activation='relu')) #128 units
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

In [46]:
#Final Layer
#it will take the pervious layer and convert into sigmoid fuction (btw 0 and 1 )
model.add(Dense(6, activation='sigmoid'))

In [47]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
  11/6981 [..............................] - ETA: 1:05:14 - loss: 0.6253

In [ ]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
##Make Predictions

In [ ]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [ ]:
df.columns[2:]

In [ ]:
batch = test.as_numpy_iterator().next()

In [ ]:
batch_X,batch_y = test.as_numpy_iterator().next()

In [80]:
model.predict(batch_X)

1/1 [==============================] - 0s 85ms/step


array([[1.45959971e-03, 1.16221294e-07, 1.53660090e-04, 1.51137319e-05,
        4.88020960e-05, 4.37866538e-06],
       [3.68381181e-04, 1.85834423e-08, 5.62152491e-05, 3.17833133e-06,
        1.06373354e-05, 6.80965968e-07],
       [8.83506262e-04, 4.42652706e-08, 9.22036124e-05, 7.43961255e-06,
        2.46741020e-05, 2.04146477e-06],
       [6.07268303e-04, 2.07262278e-08, 6.14786404e-05, 4.27952318e-06,
        1.45466829e-05, 1.12643772e-06],
       [1.25368242e-03, 8.56372324e-08, 1.31397159e-04, 1.20738659e-05,
        3.97111617e-05, 3.46846650e-06],
       [3.32964538e-03, 4.87271564e-07, 3.30046343e-04, 4.44668403e-05,
        1.48461593e-04, 1.40227949e-05],
       [5.70323085e-04, 1.76079791e-08, 5.62943751e-05, 3.84740770e-06,
        1.32758678e-05, 1.00538159e-06],
       [5.38308755e-04, 3.04481276e-08, 7.33757843e-05, 4.92075378e-06,
        1.58788225e-05, 1.13017154e-06],
       [9.82646924e-03, 3.61407456e-06, 9.58388671e-04, 1.96569890e-04,
        6.46945613e-04, 

In [81]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 96ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [90]:
res = model.predict(batch_X)

1/1 [==============================] - 0s 104ms/step


In [91]:
res

array([[1.45959971e-03, 1.16221294e-07, 1.53660090e-04, 1.51137319e-05,
        4.88020960e-05, 4.37866538e-06],
       [3.68381181e-04, 1.85834423e-08, 5.62152491e-05, 3.17833133e-06,
        1.06373354e-05, 6.80965968e-07],
       [8.83506262e-04, 4.42652706e-08, 9.22036124e-05, 7.43961255e-06,
        2.46741020e-05, 2.04146477e-06],
       [6.07268303e-04, 2.07262278e-08, 6.14786404e-05, 4.27952318e-06,
        1.45466829e-05, 1.12643772e-06],
       [1.25368242e-03, 8.56372324e-08, 1.31397159e-04, 1.20738659e-05,
        3.97111617e-05, 3.46846650e-06],
       [3.32964538e-03, 4.87271564e-07, 3.30046343e-04, 4.44668403e-05,
        1.48461593e-04, 1.40227949e-05],
       [5.70323085e-04, 1.76079791e-08, 5.62943751e-05, 3.84740770e-06,
        1.32758678e-05, 1.00538159e-06],
       [5.38308755e-04, 3.04481276e-08, 7.33757843e-05, 4.92075378e-06,
        1.58788225e-05, 1.13017154e-06],
       [9.82646924e-03, 3.61407456e-06, 9.58388671e-04, 1.96569890e-04,
        6.46945613e-04, 

In [94]:
##res.flatten()

In [87]:
###Evaluate Model

In [88]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [89]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [95]:
for batch in test.as_numpy_iterator(): ##looping each batch from our data pipeline
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()  ##flatten convert it into 1 bigg array fromate 
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 100ms/step


1/1 [==============================] - 0s 92ms/step


1/1 [==============================] - 0s 79ms/step


1/1 [==============================] - 0s 102ms/step


In [96]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8197124004364014, Recall:0.6702036261558533, Accuracy:0.47342026233673096


In [97]:
###Test and Gradio

In [99]:
##!pip install gradio jinja2

In [9]:
import gradio as gr

In [101]:
model.save('toxicity.h5')

In [4]:
model = tf.keras.models.load_model('toxicity.h5')

In [5]:
input_str = vectorizer('hey i freaken hate you!')

NameError: name 'vectorizer' is not defined

In [6]:
res = model.predict(np.expand_dims(input_str,0))

NameError: name 'input_str' is not defined

In [105]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [106]:
res

array([[0.41900614, 0.0036603 , 0.08816806, 0.0166436 , 0.1500733 ,
        0.01621035]], dtype=float32)

In [7]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [10]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

2023-04-17 17:37:01,583 [27952] WARNING  py.warnings:109: [JupyterRequire] C:\Users\avish\anaconda3\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(

2023-04-17 17:37:01,585 [27952] WARNING  py.warnings:109: [JupyterRequire] C:\Users\avish\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)

2023-04-17 17:37:01,586 [27952] WARNING  py.warnings:109: [JupyterRequire] C:\Users\avish\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)



In [ ]:
interface.launch()